In [11]:
!pip3 install tensorflow_text>=2.0.0rc0

# Model

In [12]:
import tensorflow_hub as hub
import tensorflow_text
from tqdm.notebook import tqdm
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder-multilingual/3")

# JSTS

In [13]:
import json
import pandas as pd
from urllib.request import urlopen
jsts_url = "https://raw.githubusercontent.com/yahoojapan/JGLUE/main/datasets/jsts-v1.1/valid-v1.1.json"
df = pd.DataFrame([json.loads(line) for line in urlopen(jsts_url).readlines()])
df.head(1)

,sentence_pair_id,yjcaptions_id,sentence1,sentence2,label
0,0,100312_421853-104611-31624,レンガの建物の前を、乳母車を押した女性が歩いています。,厩舎で馬と女性とが寄り添っています。,0.0


In [14]:
df.shape

(1457, 5)

## Encode

In [15]:
sentence1_embs = embed(df["sentence1"])
sentence2_embs = embed(df["sentence2"])
sentence1_embs.shape, sentence2_embs.shape

(TensorShape([1457, 512]), TensorShape([1457, 512]))

## Correlation Score

In [16]:
from scipy.spatial.distance import cosine
from scipy.stats import spearmanr
df["similarity"] = [1 - cosine(s1, s2) for s1, s2 in zip(sentence1_embs, sentence2_embs)]
spearmanr(df["similarity"], df["label"])[0]

0.7899457524478034

# JSICK

In [17]:
df = pd.read_csv("https://github.com/verypluming/JSICK/raw/main/jsick/test.tsv", sep="\t")
df.head(1)

,pair_ID,data,sentence_A_En,sentence_B_En,entailment_label_En,relatedness_score_En,corr_entailment_labelAB_En,corr_entailment_labelBA_En,sentence_A_Ja,sentence_B_Ja,entailment_label_Ja,relatedness_score_Ja,image_ID,original_caption,semtag_short,semtag_long
0,6,test,There is no boy playing outdoors and there is ...,A group of kids is playing in a yard and an ol...,neutral,3.3,NaN,NaN,戸外で遊んでいる男の子は一人もおらず、微笑んでいる男性は一人もいない,子供たちのグループが庭で遊んでいて、後ろの方には年を取った男性が立っている,contradiction,2.3,3155657768_b83a7831e5.jpg,"The children are playing outdoors , while a ma...",Negation#Numerical,"Numerical;人;名詞,接尾,助数詞,*#Negation;ない;助動詞,*,*,*#..."


In [18]:
df.shape

(4927, 16)

## Encode

In [19]:
sentence1_embs = embed(df["sentence_A_Ja"])
sentence2_embs = embed(df["sentence_B_Ja"])
sentence1_embs.shape, sentence2_embs.shape

(TensorShape([4927, 512]), TensorShape([4927, 512]))

## Correlation Score

In [20]:
from scipy.spatial.distance import cosine
from scipy.stats import spearmanr
df["similarity"] = [1 - cosine(s1, s2) for s1, s2 in zip(sentence1_embs, sentence2_embs)]
spearmanr(df["similarity"], df["relatedness_score_Ja"])[0]

0.8003486058678017